# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot

---

# Carga de datos

In [2]:
targets_competencia = pd.read_pickle("../../../data/tp2/ref_hashes_target.pkl")

In [2]:
auctions = pd.read_pickle("../../../data/tp2/auctions_tp2_formateado.pkl")
installs = pd.read_pickle("../../../data/tp2/installs_tp2_formateado.pkl")

In [3]:
# Creo columnas para luego separar
auctions['hora'] = auctions['date'].dt.hour
auctions['dia'] = auctions['date'].dt.day
auctions['dia_de_la_semana'] = auctions['date'].dt.dayofweek

installs['hora'] = installs['created'].dt.hour
installs['dia'] = installs['created'].dt.day
installs['dia_de_la_semana'] = installs['created'].dt.dayofweek

"""dias_installs = pd.get_dummies(installs["dia_de_la_semana"]).rename(columns = {0:"Lunes", 1:"Martes", 2:"Miercoles", 3:"Jueves", 4:"Viernes", 5:"Sabado", 6:"Domingo"})
installs = installs.merge(dias_installs, left_index=True, right_index=True)
del installs["dia_de_la_semana"]

dias_auctions = pd.get_dummies(auctions["dia_de_la_semana"]).rename(columns = {0:"Lunes", 1:"Martes", 2:"Miercoles", 3:"Jueves", 4:"Viernes", 5:"Sabado", 6:"Domingo"})
auctions = auctions.merge(dias_auctions, left_index=True, right_index=True)
del auctions["dia_de_la_semana"]"""

'dias_installs = pd.get_dummies(installs["dia_de_la_semana"]).rename(columns = {0:"Lunes", 1:"Martes", 2:"Miercoles", 3:"Jueves", 4:"Viernes", 5:"Sabado", 6:"Domingo"})\ninstalls = installs.merge(dias_installs, left_index=True, right_index=True)\ndel installs["dia_de_la_semana"]\n\ndias_auctions = pd.get_dummies(auctions["dia_de_la_semana"]).rename(columns = {0:"Lunes", 1:"Martes", 2:"Miercoles", 3:"Jueves", 4:"Viernes", 5:"Sabado", 6:"Domingo"})\nauctions = auctions.merge(dias_auctions, left_index=True, right_index=True)\ndel auctions["dia_de_la_semana"]'

In [6]:
auctions_competencia = auctions.loc[auctions["device_id"].isin(targets_competencia.index)].copy()

In [7]:
installs_competencia = installs.loc[installs["ref_hash"].isin(targets_competencia.index)].copy()

In [8]:
for ventana_nro in range(1,5):
    auctions_competencia["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= auctions_competencia["date"].dt.day) & ((auctions_competencia["date"].dt.day) <= (19+ventana_nro)))

In [9]:
auctions_competencia = auctions_competencia.loc[auctions['dia'] <= 23]

In [10]:
for ventana_nro in range(1,5):
    installs_competencia["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= installs_competencia["created"].dt.day) & ((installs_competencia["created"].dt.day) <= (19+ventana_nro)))

In [11]:
installs_competencia = installs_competencia.loc[installs_competencia['dia'] <= 23]

---

# Armo features

In [12]:
installs_competencia.rename(columns={"ref_hash" : "device_id", "created" : "date"}, inplace=True)
targets_competencia.rename(columns={"ref_hash" : "device_id"}, inplace=True)

/opt/anaconda/3/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [13]:
installs_competencia.sort_values(by="date", inplace=True)

/opt/anaconda/3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
features_auctions = targets_competencia.copy()

In [15]:
features_installs = targets_competencia.copy()

---

### Tiempo medio entre apariciones "descartando el primero"

In [17]:
gb = auctions_competencia.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).mean()
features_auctions["tiempo_medio_entre_apariciones_todas_las_ventanas"] = pd.Series(ps).dt.total_seconds().to_frame().rename(columns={0: "tiempo_medio_entre_apariciones_todas_las_ventanas"})

In [18]:
gb = installs_competencia.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).mean()
features_installs["tiempo_medio_entre_apariciones_todas_las_ventanas"] = pd.Series(ps).dt.total_seconds().to_frame().rename(columns={0: "tiempo_medio_entre_apariciones_todas_las_ventanas"})

---

### Función generadora de features

In [19]:
def generar_feature_en_ventanas(dataframe, generador_feature, destino):
    for ventana_nro in range(1, 5):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destino[(feature.columns[0] + "_v{}".format(ventana_nro))] = feature[feature.columns[0]]

---

### Cantidad de observaciones dentro de cada ventana

In [20]:
def cantidad_observaciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_observaciones"})

In [21]:
generar_feature_en_ventanas(auctions_competencia, cantidad_observaciones, features_auctions)

In [22]:
generar_feature_en_ventanas(installs_competencia, cantidad_observaciones, features_installs)

---

### Observaciones en cada dia

In [23]:
auctions_competencia["para_contar"] = 1
installs_competencia["para_contar"] = 1

/opt/anaconda/3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda/3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
observaciones_por_dia = auctions_competencia.groupby(["device_id", "dia"]).agg({"para_contar" : "count"}).unstack().fillna(0).stack()

In [25]:
feature = observaciones_por_dia.unstack().droplevel(axis=1, level=1)

In [26]:
feature.columns = ["dia_{}".format(x) for x in range(18,24)]

In [27]:
features_auctions = features_auctions.merge(feature, left_index=True, right_index=True, how="left")

In [28]:
observaciones_por_dia_installs = installs_competencia.groupby(["device_id", "dia"]).agg({"para_contar" : "count"}).unstack().fillna(0).stack()
feature = observaciones_por_dia_installs.unstack().droplevel(axis=1, level=1)
feature.columns = ["dia_{}".format(x) for x in range(18,24)]
features_installs = features_installs.merge(feature, left_index=True, right_index=True, how="left")

---

### Labels de ventanas anteriores

In [360]:
labels_v1_auctions = pd.read_pickle("../../modelo/targets/targets_ventana_1_auctions.pkl")
labels_v1_auctions.columns = ["label_v1"]
#labels_v2_auctions = pd.read_pickle("../../modelo/targets/targets_ventana_2_auctions.pkl")
#labels_v2_auctions.columns = ["label_v2"]
#labels_v3_auctions = pd.read_pickle("../../modelo/targets/targets_ventana_3_auctions.pkl")
#labels_v3_auctions.columns = ["label_v3"]

labels_v1_installs = pd.read_pickle("../../modelo/targets/targets_ventana_1_installs.pkl")
labels_v1_installs.columns = ["label_v1"]
#labels_v2_installs = pd.read_pickle("../../modelo/targets/targets_ventana_2_installs.pkl")
#labels_v2_installs.columns = ["label_v2"]
#labels_v3_installs = pd.read_pickle("../../modelo/targets/targets_ventana_3_installs.pkl")
#labels_v3_installs.columns = ["label_v3"]

In [361]:
features_auctions = features_auctions.merge(labels_v1_auctions, left_index=True, right_index=True, how="left")
#features_auctions = features_auctions.merge(labels_v2_auctions, left_index=True, right_index=True, how="left")
#features_auctions = features_auctions.merge(labels_v3_auctions, left_index=True, right_index=True, how="left")
features_auctions["label_v1"] = features_auctions["label_v1"].fillna(259200)
#features_auctions["label_v2"] = features_auctions["label_v2"].fillna(259200)
#features_auctions["label_v3"] = features_auctions["label_v3"].fillna(259200)

features_installs = features_installs.merge(labels_v1_installs, left_index=True, right_index=True, how="left")
#features_installs = features_installs.merge(labels_v2_installs, left_index=True, right_index=True, how="left")
#features_installs = features_installs.merge(labels_v3_installs, left_index=True, right_index=True, how="left")
features_installs["label_v1"] = features_installs["label_v1"].fillna(259200)
#features_installs["label_v2"] = features_installs["label_v2"].fillna(259200)
#features_installs["label_v3"] = features_installs["label_v3"].fillna(259200)

---

## Tiempo desde ultima aparicion en ventana hasta fin ventana

In [362]:
def tiempo_desde_ult_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana"})

In [363]:
generar_feature_en_ventanas(auctions_competencia, tiempo_desde_ult_aparicion_hasta_fin_ventana, features_auctions)
generar_feature_en_ventanas(installs_competencia, tiempo_desde_ult_aparicion_hasta_fin_ventana, features_installs)

---

## Tiempo desde primera aparición en ventana hasta fin ventana

In [364]:
def tiempo_desde_primera_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "min"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_primera_aparicion_hasta_fin_ventana"})

In [365]:
generar_feature_en_ventanas(auctions_competencia, tiempo_desde_primera_aparicion_hasta_fin_ventana, features_auctions)
generar_feature_en_ventanas(installs_competencia, tiempo_desde_primera_aparicion_hasta_fin_ventana, features_installs)

---

# Modelo

In [366]:
labels_v4_auctions = pd.read_pickle("../../modelo/targets/targets_ventana_4_auctions.pkl")
labels_v4_installs = pd.read_pickle("../../modelo/targets/targets_ventana_4_installs.pkl")

train_auctions = features_auctions.merge(labels_v4_auctions, left_index=True, right_index=True, how="left")
train_auctions["label"] = train_auctions["target"].fillna(259200)
del train_auctions["target"]

train_installs = features_installs.merge(labels_v4_installs, left_index=True, right_index=True, how="left")
train_installs["label"] = train_installs["target"].fillna(259200)
del train_installs["target"]

### Entrenamiento

In [391]:
X_auctions, y_auctions = train_auctions.iloc[:,:-1], train_auctions.iloc[:,-1]
X_train_auctions, X_test_auctions, y_train_auctions, y_test_auctions = \
    train_test_split(X_auctions, y_auctions, test_size=0.6, random_state=0)

In [392]:
model_auctions = xgb.XGBRegressor(n_jobs=6)
model_auctions.fit(X_train_auctions, y_train_auctions)
predicciones_auctions = model_auctions.predict(X_test_auctions)
rmse_auctions = np.sqrt(mean_squared_error(y_test_auctions, predicciones_auctions))
print("RMSE: %f" % (rmse_auctions))

RMSE: 71672.914245


---

In [393]:
X_installs, y_installs = train_installs.iloc[:,:-1], train_installs.iloc[:,-1]
X_train_installs, X_test_installs, y_train_installs, y_test_installs = \
    train_test_split(X_installs, y_installs, test_size=0.6, random_state=0)

In [394]:
model_installs = xgb.XGBRegressor(n_jobs=6)
model_installs.fit(X_train_installs, y_train_installs)
predicciones_installs = model_installs.predict(X_test_installs)
rmse_installs = np.sqrt(mean_squared_error(y_test_installs, predicciones_installs))
print("RMSE: %f" % (rmse_installs))

RMSE: 26263.111489


In [395]:
rmse_total = (mean_squared_error(y_test_auctions, predicciones_auctions))*len(y_test_auctions) + (mean_squared_error(y_test_installs, predicciones_installs))*len(y_test_installs)

In [396]:
rmse_total = rmse_total/(len(y_test_auctions) + len(y_test_installs))

In [397]:
rmse_total = sqrt(rmse_total)

In [398]:
print("RMSE TOTAL: %f" % (rmse_total))

RMSE TOTAL: 53975.724457


---

# Con los modelos ya entrenados, calculo los nuevos features. Me interesan las ventanas 4, 5, 6 y 7, que es hasta donde hay datos

In [206]:
auctions_competencia = auctions.loc[auctions["device_id"].isin(targets_competencia.index)].copy()

In [207]:
installs_competencia = installs.loc[installs["ref_hash"].isin(targets_competencia.index)].copy()

In [208]:
for ventana_nro in range(1,5):
    auctions_competencia["ventana_{}".format(ventana_nro)] = (((20+ventana_nro) <= auctions_competencia["date"].dt.day) & ((auctions_competencia["date"].dt.day) <= (22+ventana_nro)))

In [209]:
auctions_competencia = auctions_competencia.loc[auctions['dia'] >= 21]

In [210]:
for ventana_nro in range(1,5):
    installs_competencia["ventana_{}".format(ventana_nro)] = (((20+ventana_nro) <= installs_competencia["created"].dt.day) & ((installs_competencia["created"].dt.day) <= (22+ventana_nro)))

In [211]:
installs_competencia = installs_competencia.loc[installs_competencia['dia'] >= 21]

---

# Armo features

In [212]:
installs_competencia.rename(columns={"ref_hash" : "device_id", "created" : "date"}, inplace=True)
targets_competencia.rename(columns={"ref_hash" : "device_id"}, inplace=True)

In [213]:
installs_competencia.sort_values(by="date", inplace=True)

In [214]:
features_auctions = targets_competencia.copy()

In [215]:
features_installs = targets_competencia.copy()

---

### Tiempo medio entre apariciones "descartando el primero"

In [216]:
gb = auctions_competencia.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).mean()
features_auctions["tiempo_medio_entre_apariciones_todas_las_ventanas"] = pd.Series(ps).dt.total_seconds().to_frame().rename(columns={0: "tiempo_medio_entre_apariciones_todas_las_ventanas"})

In [217]:
gb = installs_competencia.groupby("device_id")
ps = {}
for t, g in gb:
    ps[t] = g["date"].sub(g["date"].shift()).mean()
features_installs["tiempo_medio_entre_apariciones_todas_las_ventanas"] = pd.Series(ps).dt.total_seconds().to_frame().rename(columns={0: "tiempo_medio_entre_apariciones_todas_las_ventanas"})

---

### Función generadora de features

In [218]:
def generar_feature_en_ventanas(dataframe, generador_feature, destino):
    for ventana_nro in range(1, 5):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destino[(feature.columns[0] + "_v{}".format(ventana_nro))] = feature[feature.columns[0]]

---

### Cantidad de observaciones dentro de cada ventana

In [219]:
def cantidad_observaciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_observaciones"})

In [220]:
generar_feature_en_ventanas(auctions_competencia, cantidad_observaciones, features_auctions)

In [221]:
generar_feature_en_ventanas(installs_competencia, cantidad_observaciones, features_installs)

---

### Observaciones en cada dia

In [222]:
auctions_competencia["para_contar"] = 1
installs_competencia["para_contar"] = 1

In [223]:
observaciones_por_dia = auctions_competencia.groupby(["device_id", "dia"]).agg({"para_contar" : "count"}).unstack().fillna(0).stack()

In [224]:
feature = observaciones_por_dia.unstack().droplevel(axis=1, level=1)

In [225]:
# Esta mal pero no importa, solo el nombre está mal
feature.columns = ["dia_{}".format(x) for x in range(18,24)]

In [226]:
features_auctions = features_auctions.merge(feature, left_index=True, right_index=True, how="left")

In [227]:
observaciones_por_dia_installs = installs_competencia.groupby(["device_id", "dia"]).agg({"para_contar" : "count"}).unstack().fillna(0).stack()
feature = observaciones_por_dia_installs.unstack().droplevel(axis=1, level=1)
feature.columns = ["dia_{}".format(x) for x in range(18,24)]
features_installs = features_installs.merge(feature, left_index=True, right_index=True, how="left")

---

### Labels de ventanas anteriores

In [242]:
labels_v1_auctions = pd.read_pickle("../../modelo/targets/targets_ventana_4_auctions.pkl")
labels_v1_auctions.columns = ["label_v1"]

labels_v1_installs = pd.read_pickle("../../modelo/targets/targets_ventana_4_installs.pkl")
labels_v1_installs.columns = ["label_v1"]

In [243]:
features_auctions = features_auctions.merge(labels_v1_auctions, left_index=True, right_index=True, how="left")
features_auctions["label_v1"] = features_auctions["label_v1"].fillna(259200)

features_installs = features_installs.merge(labels_v1_installs, left_index=True, right_index=True, how="left")
features_installs["label_v1"] = features_installs["label_v1"].fillna(259200)

---

## Tiempo desde ultima aparicion en ventana hasta fin ventana

In [244]:
def tiempo_desde_ult_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_aparicion_hasta_fin_ventana"})

In [245]:
generar_feature_en_ventanas(auctions_competencia, tiempo_desde_ult_aparicion_hasta_fin_ventana, features_auctions)
generar_feature_en_ventanas(installs_competencia, tiempo_desde_ult_aparicion_hasta_fin_ventana, features_installs)

---

## Tiempo desde primera aparición en ventana hasta fin ventana

In [246]:
def tiempo_desde_primera_aparicion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "min"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_primera_aparicion_hasta_fin_ventana"})

In [247]:
generar_feature_en_ventanas(auctions_competencia, tiempo_desde_primera_aparicion_hasta_fin_ventana, features_auctions)
generar_feature_en_ventanas(installs_competencia, tiempo_desde_primera_aparicion_hasta_fin_ventana, features_installs)

---

# Predicciones

In [254]:
predicciones_auctions = targets_competencia.copy()

In [256]:
predicciones_auctions["label"] = model_auctions.predict(features_auctions)

In [257]:
predicciones_installs = targets_competencia.copy()
predicciones_installs["label"] = model_installs.predict(features_installs)

In [261]:
def corregir(x):
    if x > 259200:
        return 259200
    return x

In [262]:
predicciones_auctions["label"] = predicciones_auctions["label"].apply(lambda x: corregir(x))

In [264]:
predicciones_installs["label"] = predicciones_installs["label"].apply(lambda x: corregir(x))

---

### Cargo archivo de submissions

In [266]:
submissions = pd.read_csv("../../../data/tp2/target.csv")

In [269]:
submissions.head()

,ref_hash,obj
0,1000169251625791246_sc,0
1,1000169251625791246_st,0
2,1000395625957344683_sc,0
3,1000395625957344683_st,0
4,1003027494996471685_sc,0


In [273]:
predicciones_auctions = predicciones_auctions.reset_index()

In [278]:
predicciones_auctions["ref_hash"] = predicciones_auctions["ref_hash"].apply(lambda x: str(x)+"_st")

In [279]:
predicciones_installs = predicciones_installs.reset_index()
predicciones_installs["ref_hash"] = predicciones_installs["ref_hash"].apply(lambda x: str(x)+"_sc")

In [300]:
q = submissions.copy()

In [298]:
rename_dict = predicciones_installs.set_index('ref_hash').to_dict()['label']

In [307]:
rename_dict_2 = predicciones_auctions.set_index('ref_hash').to_dict()['label']

In [306]:
q["obj"] = q["obj"].replace(rename_dict)

In [308]:
q["obj"] = q["obj"].replace(rename_dict_2)

In [314]:
q["obj"] = round(q["obj"])

In [316]:
q["obj"] = q["obj"].astype(np.int64)

In [331]:
q.set_index("ref_hash").to_csv("target.csv")